# Algoritmo de selección negativa para clasificación

### Descripción

<font size="4">  Algoritmo de selección negativa para resolver un problema de clasificación.

### Comentarios
    
- <font size="3"> Se leen los datos en formato arff.
    
- <font size="3"> Se imputan los missing values con la media (atributos numéricos) o la moda (atributos categóricos).
    
- <font size="3"> Los atributos de entrada categóricos se transforman a atributos binarios con one hot encoding.
    
- <font size="3"> Los datos de entrada y la salida se convierten en arrays numpy para mayor eficiencia en los cálculos.
    
- <font size="3"> Si el número de detectores descartados para una clase alcanza un valor máximo entonces no se generarán más detectores para esa clase. Otra altarnativa es generar una excepción en este caso, como en https://github.com/AIS-Package/aisp/.    
    
---

### Requerimientos

In [62]:
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### Lectura de los datos
#### Input:
  - $file\_path$: Nombre completo con path de la base de datos .arff a cargar
  
#### Output:
  - $X$: Atributos de entrada numéricos y categóricos
  - $y$: Salida categórica

In [63]:
def LoadData(file_path):

    data, meta = arff.loadarff(file_path)

    # Convertir a DataFrame
    df = pd.DataFrame(data)

    # Separar atributos numéricos y categóricos
    numeric_attributes = df.iloc[:, :-1] .select_dtypes(include=['int', 'float']).columns
    categorical_attributes =  df.iloc[:, :-1].select_dtypes(include=['object']).columns
    
    # Imputación para atributos numéricos
    if numeric_attributes.size>0:
        df[numeric_attributes] = SimpleImputer(strategy='mean').fit_transform(df[numeric_attributes])
        
    # Imputación para atributos categóricos
    # Aplicar hot encoding
    if categorical_attributes.size>0:
        df[categorical_attributes] = SimpleImputer(strategy='most_frequent').fit_transform(df[categorical_attributes])
        X = pd.get_dummies(data=df.iloc[:, :-1], columns=categorical_attributes)    
        X = X.astype(float)
    else:
        X = df.iloc[:, :-1]    
    
    # Salida: y
    y = [label.decode('utf-8') for label in df.iloc[:, -1]]
    class_values = meta[meta.names()[-1]][1]  # Obtiene los valores posibles de la clase 
    le = LabelEncoder()
    le.fit(class_values)
    y = le.transform(y)
        
    # Convertir las entradas y salidas en arrays numpy
    X = np.array(X)
    y = np.array(y)    
    
    L = np.min(X, axis=0)
    U = np.max(X, axis=0)
    
    return X,y,L,U

### Parámetros del algoritmo de selección negativa
- $M$: Número de detectores por cada clase
- $alpha$: Umbral de densidad
- $max\_discards$: Número máximo de dectectores descartados por cada clase
- $random\_state$: Semilla para reproducibilidad
- $L,U$: Límites de los atributos de entrada
- $X,y$: Datos

In [64]:
max_discards = 100
random_state = 1

file_path = "./DATASETS/Iris.arff"  # Atributos numéricos
# file_path = "./DATASETS/Arrhythmia.arff" # Atributos numéricos y categóricos

X,y,L,U = LoadData(file_path)

### Clase que define un nuevo estimador (BaseEstimator) entrenado con el algoritmo de selección negativa

In [65]:
class NegativeSelection(BaseEstimator):
    
    def __init__(self, M, alpha, max_discards, L, U, random_state):
        assert M>=1, "El número de detectores por clase debe ser mayor o igual que uno."
        assert alpha>0, "El umbral de densidad debe ser mayor que cero."
        assert alpha<1, "El umbral de densidad debe ser menor que uno."
        assert max_discards>=1, "El número de detectores máximo descartado por clase debe ser mayor o igual que uno."        
        self.M = M
        self.alpha = alpha
        self.max_discards = max_discards
        self.random_state = random_state
        self.L = L
        self.U = U
        
    def fit(self, X, y): # Algoritmo de selección negativa
        np.random.seed(self.random_state)
        clases = np.unique(y)
        self.X = X.tolist()
        self.y = y.tolist()
        for c in clases:
            indices_clase = np.where(y == c)[0]           
            L = np.min(X[indices_clase], axis=0)
            U = np.max(X[indices_clase], axis=0)                
            m = 0
            m_discards = 0
            while m < self.M and m_discards<self.max_discards:
                # Generar células aleatoriamente para cada clase
                b = np.random.uniform(L, U)
                
                # Solo se tienen en cuenta los detectores que no estén cerca de ningún dato - distancia Manhattan normalizada                    
                distancia = [ np.mean(np.abs((a - b) / (self.U - self.L))) for a in X ]
                if min(distancia) > self.alpha:
                    self.X.append(b)
                    self.y.append(c)
                    m += 1       
                else:
                    m_discards += 1
    
    def predict(self, X):
        z = []
        for b in X:
            distancia = [ np.mean(np.abs((a - b) / (self.U - self.L))) for a in self.X ] # DISTANCIA MANHATTAN NORMALIZADA
            i = np.argmin(distancia)
            z.append(self.y[i])
        return z
    
    def score(self, X, y):
        y_pred = self.predict(X)   
        return accuracy_score(y, y_pred)               

### Grid Search

In [66]:
def GridSearchNegativeSelection(options, n_repeat, max_discards, L, U, X, y):    
    best_score = 0
    best_options = None
    for M in options['M']:
        for alpha in options['alpha']:
            score = np.zeros(n_repeat)
            print("M:",M," alpha:",alpha)
            for i in range(n_repeat):
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)

                clasificador = NegativeSelection(M,alpha,max_discards,L,U,random_state=i)
                clasificador.fit(X_train, y_train)
                score[i] = clasificador.score(X_test, y_test)
            scoreMean = score.mean()
            print("Score Mean: ",scoreMean)
            print("")
            if scoreMean > best_score:
                best_score = scoreMean
                best_options = {'M': M, 'alpha': alpha}
    return best_score, best_options

### Ejecución del Grid Search

In [67]:
options = {'M': [10,50,100],
           'alpha': [0.01,0.1,0.5]}

n_repeat = 3
best_score, best_options = GridSearchNegativeSelection(options,n_repeat, max_discards, L, U, X, y)

print("BEST:")
print(best_score)
print(best_options)

M: 10  alpha: 0.01


Score Mean:  0.9666666666666667

M: 10  alpha: 0.1
Score Mean:  0.9666666666666667

M: 10  alpha: 0.5
Score Mean:  0.9666666666666667

M: 50  alpha: 0.01
Score Mean:  0.9444444444444445

M: 50  alpha: 0.1
Score Mean:  0.9666666666666667

M: 50  alpha: 0.5
Score Mean:  0.9666666666666667

M: 100  alpha: 0.01
Score Mean:  0.9444444444444443

M: 100  alpha: 0.1
Score Mean:  0.9666666666666667

M: 100  alpha: 0.5
Score Mean:  0.9666666666666667

BEST:
0.9666666666666667
{'M': 10, 'alpha': 0.01}


### Entrenamiento del clasificador y evaluación en el conjunto de entrenamiento

In [68]:
M = best_options['M']
alpha = best_options['alpha']

clasificador = NegativeSelection(M,alpha,max_discards,L,U,random_state=random_state)
clasificador.fit(X,y)
accTrain = clasificador.score(X,y)
print("ACC train = ",accTrain)

ACC train =  1.0


### Evaluación del clasificador en stratified k-fold cross-validation

In [69]:
n_splits=5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
resultado = cross_val_score(clasificador, X, y, scoring='accuracy',cv=cv)
print('(%d)-fold ACC = ' % n_splits, resultado)
print('(%d)-fold ACC mean (std) = %s (%s)' % (n_splits, str(resultado.mean()), str(resultado.std())))

(5)-fold ACC =  [0.96666667 0.96666667 0.93333333 0.96666667 0.9       ]
(5)-fold ACC mean (std) = 0.9466666666666667 (0.02666666666666666)


### Evaluación del clasificador en repeated stratified k-fold cross-validation

In [70]:
n_splits=5
n_repeats=2
cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
resultado = cross_val_score(clasificador, X, y, scoring='accuracy', cv=cv)
print('(%d)-fold repeat %d - ACC mean (std) = ' % (n_splits, n_repeats), resultado)
print('(%d)-fold repeat %d - ACC mean (std) = %s (%s)' % (n_splits, n_repeats, str(resultado.mean()), str(resultado.std())))

(5)-fold repeat 2 - ACC mean (std) =  [0.96666667 0.96666667 0.93333333 0.96666667 0.9        0.96666667
 0.9        1.         0.9        0.93333333]
(5)-fold repeat 2 - ACC mean (std) = 0.9433333333333334 (0.03349958540373629)
